In [1]:
# Hide all code cells (including this one) and add a toggle button
from IPython.display import HTML, display

display(HTML('''
<script>
  function toggleCodeCells() {
    var codeCells = document.querySelectorAll('div.input');
    var anyVisible = Array.from(codeCells).some(c => c.style.display !== 'none');
    codeCells.forEach(c => c.style.display = anyVisible ? 'none' : '');
  }

  // Add toggle button
  const button = document.createElement('button');
  button.textContent = 'Show/Hide Code';
  button.style.position = 'fixed';
  button.style.top = '10px';
  button.style.right = '10px';
  button.style.zIndex = 1000;
  button.style.padding = '6px 10px';
  button.style.background = '#1976D2';
  button.style.color = 'white';
  button.style.border = 'none';
  button.style.borderRadius = '6px';
  button.onclick = toggleCodeCells;
  document.body.appendChild(button);

  // Hide all code cells (including this one) on load
  document.addEventListener('DOMContentLoaded', function() {
    var firstCell = document.querySelector('div.input');
    if (firstCell) firstCell.style.display = 'none';
    toggleCodeCells();
  });
</script>
'''))



# Kelvin–Voigt Model — Viscoelastic Solid (Parallel)

The **Kelvin–Voigt model** represents a spring and a dashpot in **parallel** and captures bounded creep and no stress relaxation under a step strain.

$$
\sigma = E\,\gamma + \eta\,\dot{\gamma}, \qquad \tau = \frac{\eta}{E}
$$

- Under constant stress $ \sigma_0 $, strain approaches $ \sigma_0/E $ as $ t \to \infty $:  
  $$ \gamma(t) = \frac{\sigma_0}{E}\left(1 - e^{-t/\tau}\right) $$
- Under constant strain $ \gamma_0 $, stress is constant: $ \sigma(t) = E\,\gamma_0 $.


In [2]:

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import HBox, VBox

def creep_kv(t, E, eta, sigma0):
    tau = eta / E
    return (sigma0 / E) * (1 - np.exp(-t / tau))

def stress_relaxation_kv(t, E, eta, gamma0):
    return np.ones_like(t) * E * gamma0

# Sliders (two-column layout)
E_slider = widgets.FloatLogSlider(value=1e5, base=10, min=3, max=7, step=0.1, description='E [Pa]')
eta_slider = widgets.FloatLogSlider(value=1e6, base=10, min=4, max=8, step=0.1, description='η [Pa·s]')
sigma0_slider = widgets.FloatSlider(value=100, min=10, max=1000, step=10, description='σ₀ [Pa]')
gamma0_slider = widgets.FloatSlider(value=0.01, min=0.001, max=0.1, step=0.001, description='γ₀')
tmax_slider = widgets.FloatLogSlider(value=10, base=10, min=-2, max=4, step=0.1, description='t_max [s]')

ui_left = VBox([E_slider, eta_slider, sigma0_slider])
ui_right = VBox([gamma0_slider, tmax_slider])
ui = HBox([ui_left, ui_right])

def plot_kelvin_voigt(E, eta, sigma0, gamma0, tmax):
    tau = eta / E
    t = np.linspace(0, tmax, 400)

    gamma_creep = creep_kv(t, E, eta, sigma0)
    sigma_relax = stress_relaxation_kv(t, E, eta, gamma0)

    fig, ax = plt.subplots(1, 2, figsize=(10, 4))

    ax[0].plot(t, gamma_creep, lw=2, label='Kelvin–Voigt')
    ax[0].set_title('Creep')
    ax[0].set_xlabel('Time [s]')
    ax[0].set_ylabel('Strain γ [-]')
    ax[0].grid(True)
    ax[0].legend()

    ax[1].plot(t, sigma_relax, lw=2, label='Kelvin–Voigt')
    ax[1].set_title('Stress Relaxation')
    ax[1].set_xlabel('Time [s]')
    ax[1].set_ylabel('Stress σ [Pa]')
    ax[1].grid(True)
    ax[1].legend()

    plt.tight_layout()
    plt.show()

    print(f"Characteristic time τ = {tau:.3g} s")

out = widgets.interactive_output(
    plot_kelvin_voigt,
    {'E': E_slider, 'eta': eta_slider, 'sigma0': sigma0_slider, 'gamma0': gamma0_slider, 'tmax': tmax_slider}
)

display(ui, out)


Output()


---

## 👩‍🏫 Teacher Information (Kelvin–Voigt)

**Purpose.** Explore bounded creep and lack of stress relaxation with the Kelvin–Voigt model (parallel spring–dashpot).  
**Key relation.** $ \tau = \eta / E $. Larger $ \tau $ → slower approach to equilibrium in creep.

**Use in class.**
1. Open in Colab → *Run all*.
2. Vary $E$, $\eta$, $\sigma_0$, $\gamma_0$; observe:
   - Creep: asymptotic approach to $\sigma_0/E$.
   - Stress under step strain: constant $E\gamma_0$.
3. Compare with Maxwell: fluid-like vs solid-like behavior.
